#### <ins>Connect to the database deployed by docker</ins>

In [1]:
import mysql.connector
"""
To get hostname from MySQL deployed docker, 
run the following command 
`docker inspect -f '{{range .NetworkSettings.Networks}}{{.IPAddress}}{{end}}' <container ID>`
replace <container_ID> with the one we want to inspect. To get the id, just run `docker ps`
"""

# Establish a connection to the MySQL database
connection = mysql.connector.connect(
    # host='x.x.x.x',  # Replace with the appropriate host
    host = '172.18.0.3',
    user='audino',
    password='audino',
    database='audino',
    port=3306,
)

# Create a cursor object to execute SQL queries
cursor = connection.cursor(buffered=True)


#### <ins>Checks the data in the database</ins> 

In [2]:
table_name = "data"
# Example query
query = f"SELECT * FROM {table_name}"
cursor.execute(query)

# Fetch all rows returned by the query
column_names = [column[0] for column in cursor.description]

# Print the column names
print(column_names)

result = cursor.fetchall()

# Process the results
for row in result:
    print(row)

# Close the cursor and connection

['id', 'project_id', 'assigned_user_id', 'filename', 'original_filename', 'reference_transcription', 'is_marked_for_review', 'created_at', 'last_modified', 'youtube_end_time', 'youtube_start_time']
(1, 1, 6, '3704789cab3d4eb7a65e12bcd4dae43c.wav', 'OSR_us_000_0010_8k.wav', None, 0, datetime.datetime(2023, 6, 16, 2, 51, 47), datetime.datetime(2023, 6, 16, 7, 4, 29), None, None)
(2, 1, 4, '58558470c38f4a918494bdf83ddc2ed3.wav', '4539076_26_14_00000.wav', None, 0, datetime.datetime(2023, 6, 16, 3, 6, 23), datetime.datetime(2023, 6, 17, 5, 29, 56), None, None)
(3, 1, 5, 'ce6796c828b3448ea4781f05d35bd81d.wav', '4539076_26_14_00001.wav', None, 1, datetime.datetime(2023, 6, 16, 3, 7, 38), datetime.datetime(2023, 6, 16, 3, 7, 38), None, None)
(4, 1, 5, '7624e0549022483c888d1e586acc971c.wav', '4539076_26_14_00002.wav', None, 1, datetime.datetime(2023, 6, 16, 3, 7, 44), datetime.datetime(2023, 6, 16, 3, 7, 44), None, None)
(5, 1, 5, 'd10febc13d484ab1b1d0e034067329a1.wav', '4539076_26_14_00003.wa

#### <ins>Update the value in the selected column</ins>

##### Functions

In [3]:
def query_from_table(table_name: str, column_name: str, condition: str = "") -> any:
    """
    Queries the table according to the input column name in the database in this case, we have already use `audino`. 
    """
    query = f"SELECT {column_name} FROM {table_name}"
    query += f" WHERE {condition}" if condition else ""
    cursor.execute(query)
    queried_datas = cursor.fetchall()
    return queried_datas

def update_target_user(table_name: str, column_name: str, target_username: str, audio_ids: list[int], id_user_conversion_dict: dict) -> None:
    """
    Updates the target user based on the table name, column name, the target_username, and the id_values for update.
    """
    for id, username in id_user_conversion_dict.items():
        if username==target_username:
            for audio_id in audio_ids:
                query = f"UPDATE {table_name} SET {column_name} = {id} WHERE id = {audio_id}"
                cursor.execute(query)
                connection.commit()

#####  Checks users and their corresponding id

In [4]:

queried_datas = query_from_table("user", "id, username")
id_user_conversion_dict = {}
# Get the conversion dict for users in the database,
#  the result conversion dict will be {id: username}.
for id, username in queried_datas:
    id_user_conversion_dict[str(id)] = username

print(id_user_conversion_dict)

{'1': 'admin', '6': 'four', '3': 'one', '5': 'three', '4': 'two'}


##### Checks the transcription column in segmentation table. 

In [5]:
audio_ids = query_from_table("segmentation", "data_id, transcription")

id_transcript_dict={"id":[], "is_empty_transcript":[]}
for audio_id, transcription in audio_ids:
    id_transcript_dict["id"].append(audio_id)
    # Remove extra space and prevent the empty string to be recongnised as `already transcript`
    clean_data = transcription.strip()
    is_empty_text = len(clean_data) == 0
    id_transcript_dict["is_empty_transcript"].append(is_empty_text)
        
print(id_transcript_dict)

{'id': [6, 7, 1], 'is_empty_transcript': [False, False, True]}


##### Checks the values in the table data

In [6]:
queried_datas = query_from_table("data", "id, project_id, assigned_user_id, is_marked_for_review")

from dataclasses import dataclass

@dataclass
class DataObject:
    """ """
    audio_id: int
    project_id: int
    assigned_user: str
    is_marked_for_review: bool
    is_empty_transcript: bool

audio_id_properties_list = []
for audio_id, project_id, assigned_user, is_marked_for_review in queried_datas:
    # data_dict_template_copy = data_dict_template.copy()
    data_object = DataObject(
        audio_id=audio_id,
        project_id=project_id,
        assigned_user=id_user_conversion_dict[str(assigned_user)],
        is_marked_for_review=bool(is_marked_for_review),
        is_empty_transcript=True
    )   
    # Append values to the list
    audio_id_properties_list.append(data_object)

for i, id_value in enumerate(id_transcript_dict["id"]):
    # shift index b.c. sql starts with 1
    audio_id_properties_list[id_value-1].is_empty_transcript = id_transcript_dict["is_empty_transcript"][i]

user_properties_dict = {}
# Construct the dictionary that has a key as "assigned_user" and a value as a list of its properties.
# the list contains "audio_id", "project_id", "is_marked_for_review", and "is_empty_transcript"
for element in audio_id_properties_list:
    if element.assigned_user not in user_properties_dict:
        user_properties_dict[element.assigned_user] = []

    key = element.assigned_user
    element_dict = element.__dict__.copy()  # Get the dictionary representation of the DataObject
    del element_dict["assigned_user"]  # Remove the assigned_user attribute
    user_properties_dict[key].append(element_dict)

for username in user_properties_dict.keys():
    print(username)
    print(user_properties_dict[username])


four
[{'audio_id': 1, 'project_id': 1, 'is_marked_for_review': False, 'is_empty_transcript': True}]
two
[{'audio_id': 2, 'project_id': 1, 'is_marked_for_review': False, 'is_empty_transcript': True}, {'audio_id': 8, 'project_id': 1, 'is_marked_for_review': False, 'is_empty_transcript': True}, {'audio_id': 9, 'project_id': 1, 'is_marked_for_review': False, 'is_empty_transcript': True}, {'audio_id': 10, 'project_id': 1, 'is_marked_for_review': False, 'is_empty_transcript': True}]
three
[{'audio_id': 3, 'project_id': 1, 'is_marked_for_review': True, 'is_empty_transcript': True}, {'audio_id': 4, 'project_id': 1, 'is_marked_for_review': True, 'is_empty_transcript': True}, {'audio_id': 5, 'project_id': 1, 'is_marked_for_review': True, 'is_empty_transcript': True}]
one
[{'audio_id': 6, 'project_id': 1, 'is_marked_for_review': False, 'is_empty_transcript': False}, {'audio_id': 7, 'project_id': 1, 'is_marked_for_review': False, 'is_empty_transcript': False}]


#### Older version of the above code, please ignore it

In [9]:
# queried_datas = query_from_table("data", "id, project_id, assigned_user_id, is_marked_for_review")


# data_dict_template = \
# { 
#     "audio_id": None, 
#     "assigned_user": None, # Use with user conversion dict
#     "project_id": None, 
#     "is_marked_for_review": None,
#     "is_empty_transcript": True
# }

# audio_id_properties_list = []
# for row in queried_datas:
#     data_dict_template_copy = data_dict_template.copy()
#     # Index 1: id
#     data_dict_template_copy["audio_id"] = row[0]
#     # Index 2: project_id
#     data_dict_template_copy["project_id"] = row[1]
#     # Index 3: assigned_user_id
#     data_dict_template_copy["assigned_user"] = id_user_conversion_dict[str(row[2])]
#     # Index 4: is_marked_for_review
#     data_dict_template_copy["is_marked_for_review"] = bool(row[3])  
#     # Append values to the list
#     audio_id_properties_list.append(data_dict_template_copy)

# for i, id_value in enumerate(id_transcript_dict["id"]):
#     # shift index b.c. sql starts with 1
#     audio_id_properties_list[id_value-1]["is_empty_transcript"] = id_transcript_dict["is_empty_transcript"][i]

# user_properties_dict = {}

# # Construct the dictionary that has a key as "assigned_user" and a value as a list of its properties.
# # the list contains "audio_id", "project_id", "is_marked_for_review", and "is_empty_transcript"
# for element in audio_id_properties_list:
#     if element["assigned_user"] not in user_properties_dict:
#         user_properties_dict[element["assigned_user"]] = []

#     key = element["assigned_user"]
#     element.pop("assigned_user")
#     user_properties_dict[key].append(element)

# for element in user_properties_dict:
#     print(user_properties_dict[element])

[{'audio_id': 1, 'project_id': 1, 'is_marked_for_review': False, 'is_empty_transcript': True}]
[{'audio_id': 2, 'project_id': 1, 'is_marked_for_review': True, 'is_empty_transcript': False}, {'audio_id': 8, 'project_id': 1, 'is_marked_for_review': True, 'is_empty_transcript': False}, {'audio_id': 9, 'project_id': 1, 'is_marked_for_review': True, 'is_empty_transcript': False}, {'audio_id': 10, 'project_id': 1, 'is_marked_for_review': True, 'is_empty_transcript': False}]
[{'audio_id': 3, 'project_id': 1, 'is_marked_for_review': True, 'is_empty_transcript': False}, {'audio_id': 4, 'project_id': 1, 'is_marked_for_review': True, 'is_empty_transcript': False}, {'audio_id': 5, 'project_id': 1, 'is_marked_for_review': True, 'is_empty_transcript': False}]
[{'audio_id': 6, 'project_id': 1, 'is_marked_for_review': False, 'is_empty_transcript': False}, {'audio_id': 7, 'project_id': 1, 'is_marked_for_review': False, 'is_empty_transcript': False}]


##### Assume that we want to move the audio from user <b>"one"</b> to a new user <b>"four"</b>

In [12]:
# Conditions: 
#   1. If the `is_marked_for_review` is False and `is_empty_transcript` is None or False, we will move the video to a new user

# Steps:
# 1. Checks the total number of videos that meet the above requirements. 
# 2. Lists all of the audio ids
# 3. Update the database base on the ids and assigned_user_ids

# Step 1: We interest in user "one" 
move_from_user = "two" # We want to move the audio files from this user
move_audio_id = []

# Step 2: Lists the audio ids
for attributes in user_properties_dict[move_from_user]:
    if not(attributes["is_marked_for_review"] and attributes["is_empty_transcript"]):
        move_audio_id.append(attributes["audio_id"])

# Step 3: Update the database
update_target_user(
    table_name="data",
    column_name="assigned_user_id",
    target_username="four",
    audio_ids=move_audio_id,
    id_user_conversion_dict=id_user_conversion_dict    
)



audio id: 2, user two
audio id: 8, user two
audio id: 9, user two
audio id: 10, user two
audio id: 1, user four
audio id: 1, user four
audio id: 2, user four
audio id: 8, user four
audio id: 9, user four
audio id: 10, user four
